## Quickstart: Compare runs, choose a model, and deploy it to REST API

In this quickstart, we will:
- Run a hyperparameter tuning sweep on a training script.
- Compare the results of the runs in the MlFlow UI.
- Choose the best run and register it as a model.
- Deploy the model to a REST API.
- Build a container image suitable for deployment to a cloud platform.

![MlFlow Tracking](images/mlflow_tracking.png)

In [3]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [4]:
# Load the dataset
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=";")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [46]:
def train_valid_test_split(data):
    """Split the data into training and validation sets.
    """
    # Split the data into training, validation and test sets
    train, test = train_test_split(data, test_size=0.2, random_state=42)

    # Get training set
    train_x = train.drop(columns=["quality"], axis=1).values
    train_y = train["quality"].values.ravel()

    # Split training set into training and validation sets
    train_x, val_x, train_y, val_y = train_test_split(
        train_x,
        train_y,
        test_size=0.2,
        random_state=42,
    )

    # Get test set
    test_x = test.drop(columns=["quality"], axis=1).values
    test_y = test["quality"].values.ravel()

    return train_x, train_y, val_x, val_y, test_x, test_y

In [47]:
from keras import Input
# Create ANN model
from keras.api.models import Sequential
from keras.src.layers import Normalization
from keras.api.layers import Dense

class Model:
    def __init__(
            self,
            training_set,
            signature,
            params,
            epochs=3,
    ):
        self.data = data
        self.params = params
        self.train_x = training_set[0]
        self.train_y = training_set[1]
        self.val_x = training_set[2]
        self.val_y = training_set[3]
        self.test_x = training_set[4]
        self.test_y = training_set[5]
        self.signature = signature
        self.epochs = epochs
        self.model = self.create_model()

    def create_model(self):
        """
        Create a Sequential model with the given hyperparameters.
        :return: Compiled Keras model
        """
        # Create normalization layer first
        normalizer = Normalization()

        # Adapt the normalizer to the training data
        normalizer.adapt(self.train_x)

        # Build the model
        model = Sequential()
        model.add(Input(shape=(self.train_x.shape[1],)))
        model.add(normalizer)
        model.add(Dense(64, activation='relu'))
        model.add(Dense(1))

        model.compile(
            loss='mean_squared_error',
            metrics=[keras.metrics.RootMeanSquaredError()],
            optimizer=keras.optimizers.SGD(
                learning_rate=self.params['lr'],
                momentum=self.params['momentum'],
            ),
        )

        return model

    def track_training(self):
        """Track the training process using MLflow."""

        with mlflow.start_run(nested=True):
            self.model.fit(
                self.train_x,
                self.train_y,
                validation_data=(self.val_x, self.val_y),
                epochs=self.epochs,
                batch_size=64,
            )

            # Evaluate the model
            eval_result = self.model.evaluate(self.val_x, self.val_y, batch_size=64)
            eval_rmse = eval_result[1]

            # Log parameters and metrics
            mlflow.log_params(self.params)
            mlflow.log_metric("rmse", eval_rmse)

            # Log the model
            mlflow.keras.log_model(
                model=self.model,
                artifact_path="ann-model",
                signature=self.signature,
            )

        return {"loss": eval_rmse, "status": STATUS_OK, "model": self.model}


In [48]:
# Split the data into training, validation and test sets
split_data = train_valid_test_split(data)

# Get training features and labels
train_x = split_data[0]
train_y = split_data[1]

# Get signature
signature = infer_signature(train_x, train_y)

In [49]:
def objective(params):
    """
    Objective function for hyperparameter tuning.
    :param params: Hyperparameters
    :return: Dictionary with loss and status
    """
    model = Model(split_data, signature, params)
    return model.track_training()

In [50]:
# Define the hyperparameter space
params = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0, 1),
}
params

{'lr': <hyperopt.pyll.base.Apply at 0x2d8927170>,
 'momentum': <hyperopt.pyll.base.Apply at 0x2d8927230>}

In [53]:
# Set mlflow experiment
mlflow.set_experiment("/white_wine_quality")

with mlflow.start_run():
    # Perform hyperparameter tuning

    # Create a Trials object to store the results
    trials = Trials()

    # Perform hyperparameter tuning using Hyperopt
    best = fmin(
        fn=objective,
        space=params,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials,
    )

    # Fetch details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best hyperparameters and loss
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])

    # Log the model
    mlflow.keras.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and loss
    print(f"Best parameters: {best}")
    print(f"Best rmse: {best_run["loss"]}")

2025/04/30 08:48:07 INFO mlflow.tracking.fluent: Experiment with name '/white_wine_quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 8s 173ms/step - loss: 37.9334 - root_mean_squared_error: 6.1590
 2/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 37.7396 - root_mean_squared_error: 6.1432
 3/49 ━━━━━━━━━━━━━━━━━━━━ 0s 995us/step - loss: 37.6098 - root_mean_squared_error: 6.1327
 5/49 ━━━━━━━━━━━━━━━━━━━━ 0s 918us/step - loss: 37.7330 - root_mean_squared_error: 6.1427
 4/49 ━━━━━━━━━━━━━━━━━━━━ 0s 979us/step - loss: 37.6436 - root_mean_squared_error: 6.1354
 6/49 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - loss: 37.7937 - root_mean_squared_error: 6.1476
 7/49 ━━━━━━━━━━━━━━━━━━━━ 0s 885us/step - loss: 37.8157 - root_mean_squared_error: 6.1494
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.1777 - root_mean_squared_error: 6.0141 - val_loss: 30.9473 - val_root_mean_squared_error: 5.5630

Epoch 2/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 32.4714 - root_mean_squared_error: 5.6984
 2/49 ━━━━━━━━━━━